# Empower and CVI ICPE abstract analysis    

In [44]:
#Loading the data 

import pandas as pd
import numpy as np
from siuba import * 
# from plotnine import *  #ggplot
import os
import sys


print(os.getcwd())
# empower2019_state=pd.read_excel("./dat/2019_HHSemPOWERMapHistoricalDataset.xlsx", sheet_name="State")
# empower2019_zip=pd.read_excel("./dat/2019_HHSemPOWERMapHistoricalDataset.xlsx", sheet_name="Zip Code")
# loading in county data for 2019 through 2021 for empower 
empower2018_county=pd.read_excel("./dat/2018_HHSemPOWERMapHistoricalDataset.xlsx", sheet_name="County")
empower2019_county=pd.read_excel("./dat/2019_HHSemPOWERMapHistoricalDataset.xlsx", sheet_name="County") 
empower2020_county=pd.read_excel("./dat/2020_HHSemPOWERMapHistoricalDataset.xlsx", sheet_name="County")
empower2021_county=pd.read_excel("./dat/2021_HHSemPOWERMapHistoricalDataset.xlsx", sheet_name="County")
empower2022_county=pd.read_excel("./dat/2022_HHSemPOWERMapHistoricalDataset.xlsx", sheet_name="County")
# empower2016_state=pd.read_excel("./dat/2016_HHSemPOWERMapHistoricalDataset.xlsx", sheet_name="State")
# empower2016_county=pd.read_excel("./dat/2016_HHSemPOWERMapHistoricalDataset.xlsx", sheet_name="County") 
# empower2016_zip=pd.read_excel("./dat/2016_HHSemPOWERMapHistoricalDataset.xlsx", sheet_name="Zip Code")
# CVIclimate_county=pd.read_csv("./CVI/CVI-county-pct/CVI-county-pct-comb-climate.csv")   
cvi_extreme_events=pd.read_csv("./CVI/CVI-county-pct/CVI-county-pct-cat-CC-Extreme Events.csv")
# print(os.path.isfile("./dat/2016_HHSemPOWERMapHistoricalDataset.xlsx"))
# print(os.listdir())

c:\Users\jwan\OneDrive - University of North Carolina at Chapel Hill\RAship\place_health


## Understanding the data structures 

1. For the CVI, we are focusing on the climate change risks that directly impact communities. These extreme events are: 
   - Temperature
   - Flooding
   - Precipitation
   - Storms
   - Wildfires
   - Droughts

See guide on data wrangling https://siuba.org/guide/wrangle-joins.html


In [45]:
print(empower2022_county.head())
print(empower2022_county.dtypes)
print(cvi_extreme_events.dtypes)
print(cvi_extreme_events.head())
# print(empower2016_county.dtypes)
# print(empower2016_county.head())
# print(CVIclimate_county.dtypes)

   FIPS_Code  County_FIPS_Code    County  State_FIPS_Code State  \
0       1001                 1  Autauga                 1   AL    
1       1003                 3  Baldwin                 1   AL    
2       1005                 5  Barbour                 1   AL    
3       1007                 7     Bibb                 1   AL    
4       1009                 9   Blount                 1   AL    

   January_2022_Medicare_Benes  January_2022_Power_Dependent_Devices_DME  \
0                        10852                                       684   
1                        57238                                      2561   
2                         5841                                       272   
3                         4913                                       269   
4                         9686                                       619   

   February_2022_Medicare_Benes  February_2022_Power_Dependent_Devices_DME  \
0                         10823                               

In [52]:

empower2022_county['State'].value_counts()
empower2022_county['State'].value_counts(normalize=True)
empower2022_county>>filter(_.State.str.contains("NC"))>>group_by(_.County_FIPS_Code)
empower2022_county>>count(_.State)


empower_nc = empower2022_county>>filter(_, _.State.str.contains("NC"))
empower_nc.describe()
empower_nc['County_FIPS_Code'].value_counts()
# empower2022_county.describe()



County_FIPS_Code
1      1
127    1
147    1
145    1
143    1
      ..
61     1
59     1
57     1
55     1
199    1
Name: count, Length: 100, dtype: int64

In [56]:
cvi_extreme_events.dtypes

cvi_nc_ee = cvi_extreme_events>>filter(_.Name.str.contains("NC"))
# I want to join on the FIPs code 
cvi_nc_ee.FIPS
empower_nc.FIPS_Code


cvi_nc_ee.columns
cvi_nc_ee.head()


,row#,Name,FIPS,Source,ToxPi Score,Temperature,Droughts,Wildfires,Precipitation,Flooding,Storms
1890,1891,"NC, Alamance",37001,"+36.0419739,-079.3999353",0.441518,0.358784,0.628455,0.211854,0.622625,0.209777,0.617613
1891,1892,"NC, Alexander",37003,"+35.9209506,-081.1774668",0.450383,0.289138,0.785691,0.339865,0.491695,0.051361,0.744550
1892,1893,"NC, Alleghany",37005,"+36.4893563,-081.1322988",0.535352,0.424692,0.752846,0.456633,0.635002,0.137720,0.805221
1893,1894,"NC, Anson",37007,"+34.9749961,-080.1099594",0.426051,0.277265,0.556098,0.335047,0.366410,0.234482,0.787005
1894,1895,"NC, Ashe",37009,"+36.4363053,-081.4987699",0.582109,0.477078,0.793333,0.437038,0.635002,0.282158,0.868044


In [ ]:
# empower_nc.columns

nc2022_ee=cvi_nc_ee>>left_join(_, empower_nc, {"FIPS":"FIPS_Code"})
nc2022_ee.describe()
nc2022_ee.set_index('FIPS_Code', inplace=True)

# tmp = nc2022_ee>>mutate(county= _.Name.str.split(expand=True)[1])

nc2022_ee.Name.str.split(expand=True)

# Getting the attributes and methods associated with nc2022_ee
print([m for m in dir(nc2022_ee) if not m.startswith("__")])
# print(dir(nc2022_ee)) 


In [48]:

# Sorting data by CVI score 
# Sorting data by CVI score 
nc2022_ee>>select(_.County, _.FIPS, _.Temperature, _.Droughts, _.Wildfires, _.Precipitation, _.Flooding, _.Storms, _.January_2022_Medicare_Benes)>>arrange(-_.Temperature)>>_.head(10)
nc2022_ee>>select(_.County, _.FIPS, _.Temperature, _.Droughts, _.Wildfires, _.Precipitation, _.Flooding, _.Storms, _.January_2022_Medicare_Benes)>>arrange(-_.Flooding)>>_.head(10)
nc2022_ee>>select(_.County, _.FIPS, _.Temperature, _.Droughts, _.Wildfires, _.Precipitation, _.Flooding, _.Storms, _.January_2022_Medicare_Benes)>>arrange(-_.Storms)>>_.head(10)
nc2022_ee>>select(_.County, _.FIPS, _.Temperature, _.Droughts, _.Wildfires, _.Precipitation, _.Flooding, _.Storms, _.January_2022_Medicare_Benes)>>arrange(-_.Precipitation)>>_.head(10)
nc2022_ee>>select(_.County, _.FIPS, _.Temperature, _.Droughts, _.Wildfires, _.Precipitation, _.Flooding, _.Storms, _.January_2022_Medicare_Benes)>>arrange(-_.January_2022_Medicare_Benes)>>_.head(10)
# (nc2022_ee>>select(_.County, _.FIPS, _.Temperature, _.Droughts, _.Wildfires, _.Precipitation, _.Flooding, _.Storms, _.January_2022_Medicare_Benes)>>arrange(-_.Temperature)>>_.head(10)).to_csv("top10_temp.csv")

(nc2022_ee
    >> mutate(January_DME_proportion=_.January_2022_Power_Dependent_Devices_DME/_.January_2022_Medicare_Benes)
    >> select(_.County, _.FIPS, _.Temperature, _.Droughts, _.Wildfires, _.Precipitation, _.Flooding, _.Storms, _.January_2022_Medicare_Benes, _.January_2022_Power_Dependent_Devices_DME, _.January_DME_proportion)
    #>> filter(_.County.str.contains("Robeson"))
    >> arrange(-_.January_DME_proportion)
    >> _.head(10)
)

,County,FIPS,Temperature,Droughts,Wildfires,Precipitation,Flooding,Storms,January_2022_Medicare_Benes,January_2022_Power_Dependent_Devices_DME,January_DME_proportion
FIPS_Code,,,,,,,,,,,
37193,Wilkes,37193,0.306124,0.819350,0.366849,0.635002,0.241958,0.709409,16694,1440,0.086259
37009,Ashe,37009,0.477078,0.793333,0.437038,0.635002,0.282158,0.868044,8087,692,0.085569
37171,Surry,37171,0.342568,0.686992,0.315130,0.619477,0.258862,0.690906,19586,1631,0.083274
37005,Alleghany,37005,0.424692,0.752846,0.456633,0.635002,0.137720,0.805221,3609,298,0.082571
37075,Graham,37075,0.443876,0.800976,0.509476,0.547498,0.060354,0.858721,2167,171,0.078911
37011,Avery,37011,0.476473,0.825203,0.465467,0.629682,0.199808,0.885399,5751,452,0.078595
37047,Columbus,37047,0.137368,0.383252,0.535496,0.308978,0.474152,0.755737,13183,1016,0.077069
37027,Caldwell,37027,0.291282,0.784878,0.345165,0.491695,0.284109,0.703959,19483,1472,0.075553
37121,Mitchell,37121,0.433872,0.817886,0.501927,0.562371,0.150940,0.952955,4463,328,0.073493


In [51]:
# Getting the baseline combined 
cvi_bl= pd.read_csv("./CVI/CVI-county-pct/CVI-county-pct-comb-baseline.csv")
cvi_bl.columns= cvi_bl.columns.str.replace('[. ]', "_", regex=True)
cvi_bl.columns
cvi_blnc=cvi_bl>>filter(_.Name.str.contains("NC"))
(cvi_blnc>>arrange(-_.ToxPi_Score))>>_.head(10)
(cvi_blnc>>arrange(-_.Baseline_SocialEconomic))>>_.head(10)
(cvi_blnc>>arrange(-_.Baseline_Health))>>_.head(10)
(cvi_blnc>>arrange(-_.Baseline_Infrastructure))>>_.head(10)
(cvi_blnc>>arrange(-_.Baseline_Environment))>>_.head(10)
cvi_blnc=(cvi_bl>>filter(_.Name.str.contains("NC"))
>>rename(CVI_baseline_score=_.ToxPi_Score))
cvi_blnc.columns


,row#,Name,FIPS,Source,ToxPi_Score,Baseline_Health,Baseline_SocialEconomic,Baseline_Infrastructure,Baseline_Environment
1967,1968,"NC, Robeson",37155,"+34.6392096,-079.1008811",0.767540,0.895961,0.769296,0.829176,0.575726
1972,1973,"NC, Scotland",37165,"+34.8400226,-079.4773366",0.764246,0.826554,0.885464,0.738502,0.606464
1966,1967,"NC, Richmond",37153,"+35.0019574,-079.7478094",0.725785,0.813240,0.774070,0.756829,0.558999
1931,1932,"NC, Halifax",37083,"+36.2514376,-077.6448423",0.719222,0.847080,0.815518,0.701087,0.513204
1893,1894,"NC, Anson",37007,"+34.9749961,-080.1099594",0.711657,0.788223,0.680009,0.756040,0.622354
1922,1923,"NC, Edgecombe",37065,"+35.9170554,-077.6026551",0.694696,0.796622,0.793602,0.778534,0.410025
1943,1944,"NC, Lenoir",37107,"+35.2380617,-077.6390233",0.690024,0.818304,0.836712,0.622044,0.483036
1913,1914,"NC, Columbus",37047,"+34.2604711,-078.6363776",0.687139,0.825180,0.788055,0.671138,0.464183
1912,1913,"NC, Cleveland",37045,"+35.3346302,-081.5571145",0.687077,0.727378,0.637406,0.720759,0.662765
1898,1899,"NC, Bladen",37017,"+34.5919485,-078.5395125",0.683480,0.778051,0.828761,0.750066,0.377041


In [57]:
# Getting Climate combined 
cvi_cc= pd.read_csv( "./CVI/CVI-county-pct/CVI-county-pct-comb-climate.csv")
cvi_cc.columns= cvi_cc.columns.str.replace('[. ]', '_', regex=True)
cvi_ccnc = cvi_cc>>filter(_.Name.str.contains("NC"))
(cvi_ccnc>>arrange(-_.ClimateChange_ExtremeEvents))>>_.head(10)
(cvi_ccnc>>arrange(-_.ToxPi_Score))>>_.head(10)
# (cvi_ccnc>>arrange(-_.ToxPi_Score)
#     >>filter(_.Name.str.contains("Robeson")))

cvi_ccnc = (cvi_cc>>filter(_.Name.str.contains("NC"))
>>rename(CVI_climate_score=_.ToxPi_Score))

cvi_ccnc.columns
cvi_ccnc.head()

,row#,Name,FIPS,Source,CVI_climate_score,ClimateChange_Health,ClimateChange_SocialEconomic,ClimateChange_ExtremeEvents
1890,1891,"NC, Alamance",37001,"+36.0419739,-079.3999353",0.519006,0.807331,0.409614,0.340073
1891,1892,"NC, Alexander",37003,"+35.9209506,-081.1774668",0.573600,0.689689,0.670931,0.360179
1892,1893,"NC, Alleghany",37005,"+36.4893563,-081.1322988",0.589798,0.721755,0.494758,0.552883
1893,1894,"NC, Anson",37007,"+34.9749961,-080.1099594",0.600868,0.831048,0.666560,0.304995
1894,1895,"NC, Ashe",37009,"+36.4363053,-081.4987699",0.598443,0.687504,0.448902,0.658923


In [43]:
cvi_all = pd.read_csv("./CVI/CVI-county-pct/CVI-county-pct-comb.csv")
dir(cvi_all)
cvi_all.columns= cvi_all.columns.str.replace('[. ]', '_', regex=True)
dir(cvi_all)
cvi_allnc = cvi_all>>filter(_.Name.str.contains("NC"))
cvi_allnc>>arrange(-_.Baseline_Infrastructure, -_.Baseline_Health)>>_.head(10)
cvi_allnc>>arrange(-_.ClimateChange_Health)>>_.head(10)
cvi_allnc>>arrange(-_.ClimateChange_SocialEconomic)>>_.head(10)
cvi_allnc>>arrange(-_.ToxPi_Score)>>_.head(10)

cvi_allnc = (cvi_all>>filter(_.Name.str.contains("NC")) 
            >>rename(CVI_overall_score=_.ToxPi_Score))
cvi_allnc.columns


,row#,Name,FIPS,Source,ToxPi_Score,Baseline_Health,Baseline_SocialEconomic,Baseline_Infrastructure,Baseline_Environment,ClimateChange_Health,ClimateChange_SocialEconomic,ClimateChange_ExtremeEvents
1972,1973,"NC, Scotland",37165,"+34.8400226,-079.4773366",0.684240,0.826554,0.885464,0.738502,0.606464,0.817759,0.686048,0.228888
1967,1968,"NC, Robeson",37155,"+34.6392096,-079.1008811",0.669403,0.895961,0.769296,0.829176,0.575726,0.795568,0.596224,0.223870
1931,1932,"NC, Halifax",37083,"+36.2514376,-077.6448423",0.664229,0.847080,0.815518,0.701087,0.513204,0.806187,0.535776,0.430752
1893,1894,"NC, Anson",37007,"+34.9749961,-080.1099594",0.664176,0.788223,0.680009,0.756040,0.622354,0.831048,0.666560,0.304995
1898,1899,"NC, Bladen",37017,"+34.5919485,-078.5395125",0.659026,0.778051,0.828761,0.750066,0.377041,0.739296,0.787114,0.352855
1966,1967,"NC, Richmond",37153,"+35.0019574,-079.7478094",0.652215,0.813240,0.774070,0.756829,0.558999,0.792232,0.626652,0.243485
1897,1898,"NC, Bertie",37015,"+36.0613302,-076.9623670",0.650235,0.710433,0.742936,0.723674,0.432412,0.701088,0.737094,0.504010
1971,1972,"NC, Sampson",37163,"+34.9905745,-078.3713822",0.639837,0.753952,0.814295,0.678326,0.459482,0.748427,0.654708,0.369666
1955,1956,"NC, Northampton",37131,"+36.4217739,-077.3983518",0.636029,0.822344,0.795329,0.622289,0.420747,0.773630,0.589239,0.428623
1983,1984,"NC, Washington",37187,"+35.8445891,-076.5723342",0.635996,0.752026,0.675675,0.779071,0.364421,0.699484,0.793158,0.388134


In [62]:
# https://empowerprogram.hhs.gov/about-empowermap.html

### Testing calculations--for normalized column data 

In [59]:
0.2969916+  0.7422759+            0.6314711+            0.8042630             +       0.5342240 +0.5295359            +   0.4560483
3.9948097999999996/7

0.5706871142857143

In [61]:
(0.4225221-0.2915693) /(0.7325002-0.2915693)

0.2969916601444807